Ai club task for round 2

In [1]:
import kagglehub

path = kagglehub.dataset_download("orvile/ravdess-dataset")
print("Path to dataset files:", path)


Path to dataset files: /Users/myash/.cache/kagglehub/datasets/orvile/ravdess-dataset/versions/1


In [2]:
'''IMPORTING ALL THE LIBRARIES WE WILL USE'''
import librosa
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from itertools import cycle
import glob
import librosa.display
import IPython.display as ipd
import csv
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
sns.set_theme(style = "white", palette = None)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])
audio_files = glob.glob(
    f"{path}/**/Audio_Speech_Actors_01-24/**/*.wav",
    recursive=True
)

print(len(audio_files))




1440


In [3]:
# """Tesing Zone
# Here will test code for individual sample or small group
# """
# y, sr = librosa.load(audio_files[i])
# print(f'y: {y[:10]}')
# print(f'shape y: {y.shape}')
# print(sr)

# pd.Series(y).plot(figsize = (10, 5), 
#                 lw = 1, 
#                 title = 'Raw Audio Example',
#                 color = color_pal[0])
# D = librosa.stft(y)
# S_db = librosa.amplitude_to_db(np.abs(D), ref = np.max)
# S_db.shape
# fig, ax = plt.subplots(figsize = (10, 5))
# img = librosa.display.specshow(S_db,
#                             x_axis = 'time',
#                             y_axis = 'log',
#                             ax = ax)
# #MEL Spectogram
# S = librosa.feature.melspectrogram(y = y, sr = sr, n_mels = 128)
# S_db_mel = librosa.amplitude_to_db(S, ref = np.max)
# img_mel = librosa.display.specshow(S_db_mel,
#                             x_axis = 'time',
#                             y_axis = 'log',
#                             ax = ax)


# for i in range(3):  # keep small
#     y, sr = librosa.load(audio_files[i], sr=22050)
#     y, _ = librosa.effects.trim(y)

#     print(f"File {i} | shape: {y.shape} | sr: {sr}")

#     # Waveform
#     fig, ax = plt.subplots(figsize=(10, 3))
#     ax.plot(y, linewidth=1)
#     ax.set_title(f"Waveform {i}")
#     plt.show()
#     plt.close()

#     # Spectrogram
#     D = librosa.stft(y)
#     S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

#     fig, ax = plt.subplots(figsize=(10, 4))
#     img = librosa.display.specshow(
#         S_db, x_axis='time', y_axis='log', ax=ax
#     )
#     fig.colorbar(img, ax=ax, format="%+2.0f dB")
#     ax.set_title(f"Spectrogram {i}")
#     plt.show()
#     plt.close()


# #MEL Spectogram
# librosa.feature.melspectogram(y, sr = sr, n_mels = 128)

In [4]:
'''
Phase 1: Preprocessing & EDA
'''

#AUDIO CLEANING
def trim_silence(audio):
    """
    Removes silence from the beginning and end of an audio signal.
    """
    trimmed_audio, _ = librosa.effects.trim(audio, top_db=20)
    return trimmed_audio


print("Completed Audio silence")

#Data Augmentation Functions
def add_white_noise(data):
    """
    Adds small random white noise to the signal.
    """
    noise = 0.002 * np.random.randn(len(data))
    return data + noise
print("Completed Adding white Noise")

def pitch_shift(data, sr):
    """
    Shifts pitch upward by 2 semitones.
    """
    return librosa.effects.pitch_shift(data, sr=sr, n_steps=2)
print("Completed Pitch shift")
#Mel- Log Scale
def extract_log_mel_spectrogram(audio, sr):
    """
    Converts audio into a log-scaled Mel-Spectrogram.
    """
    # Generate Mel-Spectrogram (power scale)
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels = 128)

    # Convert power spectrogram to decibel (log) scale
    log_mel_spec = librosa.power_to_db(mel_spec, ref = np.max, top_db = 80)

    return log_mel_spec

print("Completed extraction of log_mel_spectogram")

#Padding Function
def pad_spectrogram(spec, max_frames):
    if spec.shape[1] < max_frames:
        pad_width = max_frames - spec.shape[1]
        spec = np.pad(
            spec,
            ((0, 0), (0, pad_width)),
            mode="constant",
            constant_values=-80.0  # Change spec.min() to -80.0
        )
    else:
        spec = spec[:, :max_frames]
    return spec
def normalize_spec(spec):
    return (spec + 80) / 80   # scale from [-80,0] → [0,1]

print("Completed pad_spectogram")
frame_lengths = []

for file in audio_files:
    audio, sr = librosa.load(file, sr=22050)
    audio = trim_silence(audio)
    mel = librosa.feature.melspectrogram(y=audio, sr=sr)
    frame_lengths.append(mel.shape[1])

MAX_FRAMES = int(np.percentile(frame_lengths, 95))

print("MAX_FRAMES selected:", MAX_FRAMES)


#Full Processing for a single audio file
def process_audio_file(file_path):
    """
    Load → Trim → Augment → Log-Mel → Pad
    """
    audio, sr = librosa.load(file_path, sr=22050)
    audio = trim_silence(audio)

    # Original
    spec_orig = extract_log_mel_spectrogram(audio, sr)
    spec_orig = pad_spectrogram(spec_orig, MAX_FRAMES)
    spec_orig = normalize_spec(spec_orig)
   

    # White noise
    audio_noise = add_white_noise(audio)
    spec_noise = extract_log_mel_spectrogram(audio_noise, sr)
    spec_noise = pad_spectrogram(spec_noise, MAX_FRAMES)
    spec_noise = normalize_spec(spec_noise)

    # Pitch shift
    audio_pitch = pitch_shift(audio, sr)
    spec_pitch = extract_log_mel_spectrogram(audio_pitch, sr)
    spec_pitch = pad_spectrogram(spec_pitch, MAX_FRAMES)
    spec_pitch = normalize_spec(spec_pitch)

    return spec_orig, spec_noise, spec_pitch
print("Completed full batch audio processing")

#Batch Processing for a dataset folder

features = []
labels = []

for file in audio_files:
    # RAVDESS emotion label
    label = file.split("/")[-1].split("-")[2]

    specs = process_audio_file(file)

    for spec in specs:
        features.append(spec)
        labels.append(label)


#Getting CNN ready tensors
X = np.array(features)
y = np.array(labels)

# Add channel dimension for CNNs
X = X[..., np.newaxis]

print("X shape:", X.shape)  # (samples, mel, time, 1)
print("y shape:", y.shape)

# for i in range(10):
# #Testing on One File
#     plt.figure(figsize=(10, 4))
#     librosa.display.specshow(
#         X[i].squeeze(), 
#         sr=22050, 
#         x_axis='time', 
#         y_axis='mel', 
#         cmap='magma', 
#         vmin=-80, 
#         vmax=0
#     )
#     plt.colorbar(format="%+2.0f dB")
#     plt.title("Log-Mel Spectrogram")
#     plt.tight_layout()
#     plt.show()





Completed Audio silence
Completed Adding white Noise
Completed Pitch shift
Completed extraction of log_mel_spectogram
Completed pad_spectogram
MAX_FRAMES selected: 105
Completed full batch audio processing
X shape: (4320, 128, 105, 1)
y shape: (4320,)


In [6]:
"""
Starting to develop CNN artitecture.
"""



le = LabelEncoder()
y_int = le.fit_transform(y)
y_cat = tf.keras.utils.to_categorical(y_int, num_classes=8)

X_train, X_temp, y_train, y_temp, y_int_train, y_int_temp = train_test_split(
    X, y_cat, y_int, test_size=0.2, stratify=y_int, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_int_temp, random_state=42
)


input_shape = X_train.shape[1:]   # (128, MAX_FRAMES, 1)


#Prepare data for training




#Create a convolutional neural network
model = tf.keras.models.Sequential([

    #Convoluthional layer. Learn 32 filters using a 3x3 kernel
    tf.keras.layers.Conv2D(32, (3,3), activation = "relu", input_shape = input_shape),
    tf.keras.layers.BatchNormalization(),
    #Max-pooling layer, using 2x2 pool size
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),


    tf.keras.layers.Conv2D(64, (3,3), activation = "relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2)),


    tf.keras.layers.Conv2D(128, (3,3), activation = "relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2)),
    #Flatten units
    tf.keras.layers.Flatten(),

    #Add a hidden layer with dropout
    tf.keras.layers.Dense(256, activation = "relu"),
    tf.keras.layers.Dropout(0.5),

    #Add an output layer units for all 10 digits
    tf.keras.layers.Dense(8, activation = "softmax")
])

model.compile(
    optimizer = "adam",
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
)

model.fit(
    X_train, y_train,
    validation_data=(X_temp, y_temp),
    epochs=10,
    batch_size=32
)

#Evaluate neural network performace
model.evaluate(X_test, y_test, verbose = 2)




/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 33s 290ms/step - accuracy: 0.1869 - loss: 3.1938 - val_accuracy: 0.1331 - val_loss: 28.9255
Epoch 2/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 30s 278ms/step - accuracy: 0.2031 - loss: 2.1477 - val_accuracy: 0.1331 - val_loss: 30.0473
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 30s 280ms/step - accuracy: 0.2289 - loss: 2.0185 - val_accuracy: 0.1343 - val_loss: 26.1771
Epoch 4/10
 71/108 ━━━━━━━━━━━━━━━━━━━━ 10s 276ms/step - accuracy: 0.2260 - loss: 1.9641

KeyboardInterrupt: 